# LangChain 으로 RAG Agent 구축
LLM이 지원하는 가장 강력한 애플리케이션 중 하나는 정교한 질의응답(Q&A) 챗봇입니다. 이는 특정 출처 정보에 대한 질문에 답할 수 있는 애플리케이션입니다. 이러한 애플리케이션은 검색 증강 생성(Retrieval Augmented Generation, RAG) 이라는 기술을 사용합니다.  

우리는 다음과 같은 개념을 다룰 것입니다:
- 인덱싱(Indexing) : 소스에서 데이터를 수집하고 인덱싱하는 파이프라인입니다. 일반적으로 별도의 프로세스로 진행됩니다.
- 검색 및 생성(Retrieval and Generation) : 런타임에 사용자 쿼리를 받아 인덱스에서 관련 데이터를 검색한 다음 이를 모델에 전달하는 실제 RAG 프로세스입니다.

### **인덱싱 단계**  
일반적인 데이터 인덱싱 과정은 다음과 같습니다:  

1. **로드(Load)**  
   - 먼저 데이터를 불러와야 합니다. 이는 문서 로더(Document Loaders)를 사용하여 수행됩니다.  

2. **분할(Split)**  
   - 텍스트 분할기(Text Splitters)를 사용해 큰 `문서(Document)`를 작은 청크(chunk)로 나눕니다.  
   - 이렇게 하면 검색이 더 효율적이며, 모델의 제한된 컨텍스트 윈도우에 맞출 수 있습니다.  

3. **저장(Store)**  
   - 분할된 데이터를 저장하고 인덱싱할 장소가 필요합니다.  
   - 일반적으로 벡터 스토어(VectorStore)와 임베딩 모델(Embeddings)을 사용합니다.  
---

### **검색 및 생성 단계**  

일반적인 검색 및 생성 과정은 다음과 같습니다:  

4. **검색(Retrieve)**  
   - 사용자 입력을 받아 검색기(Retriever)를 사용하여 저장된 데이터에서 관련 청크를 검색합니다.  

5. **생성(Generate)**  
   - 챗 모델(ChatModel) 또는 LLM이 검색된 데이터를 포함한 프롬프트를 사용해 답변을 생성합니다.  

In [ ]:
# model = init_chat_model("gpt-5-nano", model_provider="openai")
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
# Hugging Face Hub에서 한국어 임베딩 모델 로드
# KURE-v1은 한국어에 특화된 문장 임베딩 모델입니다.

In [ ]:
# InMemoryVectorStore - 메모리 내에서 벡터 데이터를 저장 및 빠른 검색

이  노트북에서는 **웹사이트 콘텐츠에 대한 질문에 답변하는 애플리케이션**을 구축합니다.  
**텍스트를 로드, 분할, 인덱싱**한 후, **사용자 질문을 기반으로 관련 데이터를 검색**하고 답변을 생성합니다.

## **단계별 상세 설명**

## **1. 인덱싱 (Indexing)**

### **문서 불러오기 (Loading Documents)**

먼저 **Document Loaders** 중 **[WebBaseLoader](https://python.langchain.com/docs/integrations/document_loaders/web_base/)** 를 사용하여 블로그 게시물의 내용을 불러옵니다. 

* `WebBaseLoader`클라스는 내부적으로 `urllib`을 사용해 **웹 URL에서 HTML을 로드**합니다.
* 이후, `BeautifulSoup`을 사용해 **텍스트로 파싱**하고 **Document** 객체 목록으로 반환합니다.

#### **HTML → 텍스트 변환 커스터마이징**

* 우리는 `<h1>`, `<h2>`, `<h3>`, `<p>`, `<pre>`, `<li>` 등 주요 콘텐츠를 포함하는 태그만 추출하도록 설정합니다.
* 또한 일부 웹사이트에서는 User-Agent를 설정하지 않으면 콘텐츠를 차단할 수 있으므로, `requests_kwargs`를 사용해 User-Agent를 지정해줍니다.

In [ ]:
# 주요 콘텐츠 태그만 필터링 (제목, 본문, 코드 등)
# WebBaseLoader 사용: requests_kwargs로 User-Agent 설정
# 문서 로드
# 결과 확인

In [ ]:
# 문서 길이
# 첫번째 500 자 출력


---

### **문서 분할 (Splitting documents)**  

문서 길이가 언어 모델의 **컨텍스트 윈도우(context window)** 에 넣기에 너무 길 경우, 너무 긴 입력은 **정보를 효과적으로 찾아내기 어려울 수 있습니다.**  

이 문제를 해결하기 위해, **`Document`를 작은 청크(chunk)로 분할**하여 **임베딩(embedding)** 및 **벡터 저장(vector storage)** 에 사용합니다.  
이렇게 하면 블로그 게시물의 **가장 관련성 높은 부분만 검색**할 수 있습니다.  

---

**RecursiveCharacterTextSplitter**는 문서를 **공통 구분자(예: 줄바꿈)** 를  사용해 재귀적으로 분할합니다.  일반적인 텍스트 사용 사례에 가장 적합한 텍스트 분할기입니다.

In [ ]:
# 불러온 문서를 설정한 기준에 따라 청크로 분할
# 분할된 청크(서브 문서)의 개수 출력

### **문서 저장 (Storing documents)**

이제 분할된 **텍스트 청크**를 인덱싱해야 합니다. 이를 통해 검색할 수 있습니다.  

1. 각 **문서 청크**의 내용을 **임베딩(embedding)** 합니다.
2. 이 **임베딩을 벡터 스토어(Vector Store)** 에 삽입합니다.


In [ ]:
# 분할된 문서 청크(all_splits)는 임베딩되어 벡터 스토어에 저장됩니다.
# 반환값은 저장된 각 문서 청크의 고유 ID 목록입니다.
# 첫 세 개의 문서 ID를 출력합니다.

이로써 **인덱싱(Indexing)** 단계가 완료되었습니다!

- 이제 우리는 **질의 가능한 벡터 스토어**를 보유하고 있습니다.  
- 게시물의 청크가 저장되어 있으며, 사용자 질문을 받으면 **관련 청크를 반환**할 수 있습니다.  

---

## **2. 검색 및 생성 (Retrieval and Generation)**

이제 실제 **애플리케이션 로직(application logic)** 을 작성하여 다음과 같은 작업을 수행할 것입니다:  

1. **사용자 질문을 입력받기**  
2. **질문과 관련된 문서 청크 검색**  
3. **검색된 문서와 질문을 모델에 전달**  
4. **챗 모델(Chat Model)이 답변을 생성**  
---

### **RAG Agent**  
RAG 애플리케이션의 한 가지 형태는 정보를 검색하는 도구를 갖춘 간단한 에이전트 입니다. 벡터 저장소를 래핑하는 도구를 구현하여 최소 RAG 에이전트를 구성할 수 있습니다 .

In [ ]:
def retrieve_context(query: str):
    # 벡터 스토어(Vector Store)에서 유사한 문서 2개 검색
    # 검색된 문서들을 문자열로 직렬화 (출처 정보 + 본문)
    # 문자열(serialized)과 실제 문서 객체 리스트(retrieved_docs)를 함께 반환

In [ ]:
# 사용할 도구 목록 정의 (retrieve_context 도구 포함)
# 필요하다면 사용자 정의 프롬프트(시스템 지침) 설정
# 에이전트 생성: 모델, 도구, 시스템 프롬프트를 결합하여 구성

In [ ]:
# 에이전트에게 사용자 메시지를 스트리밍 방식으로 전달
    # 마지막 메시지(모델의 응답)를 예쁘게 출력

-----------------------------
또 다른 일반적인 접근 방식은 2단계 체인으로, 항상 검색을 실행하고(원시 사용자 쿼리를 사용할 수도 있음) 그 결과를 단일 LLM 쿼리의 컨텍스트로 통합하는 방식입니다. 이렇게 하면 쿼리당 추론 호출이 한 번만 발생하여 지연 시간이 단축되는 반면, 유연성은 저하됩니다.
이 접근 방식에서는 더 이상 루프에서 모델을 호출하지 않고 대신 단일 패스를 만듭니다.
에이전트에서 도구를 제거하고 대신 검색 단계를 사용자 정의 프롬프트에 통합하여 이 체인을 구현할 수 있습니다.

In [ ]:
def prompt_with_context(request: ModelRequest) -> str:
    # 사용자의 마지막 질의 텍스트 가져오기
    # 벡터 스토어(Vector Store)에서 마지막 질의와 유사한 문서 검색
    # 검색된 문서의 본문 내용을 하나의 문자열로 결합
    # 시스템 프롬프트(system message)에 문맥 추가
# 미들웨어(middleware)를 통해 문맥 주입 기능이 포함된 에이전트 생성